To apply hierarchical clustering and k-means clustering to a tabular dataset with categorical variables, the following steps can be taken:

1. Load the data into a pandas dataframe.
2. Preprocess the data by converting the categorical variables into one-hot encoded vectors using the `pd.get_dummies()` function.
3. Apply the clustering algorithm of choice, such as hierarchical clustering or k-means clustering, to the preprocessed data.
4. Analyze the resulting clusters, such as by visualizing them using t-SNE or by examining the characteristics of the data points in each cluster.
5. (Optional) Use the resulting clusters to make predictions or to guide further analysis of the data.

It's important to note that the choice of clustering algorithm and the preprocessing steps may vary depending on the specific dataset and research question at hand. It's also important to carefully choose the number of clusters in order to avoid overfitting or underfitting the data. Various methods, such as the elbow method or silhouette analysis, can be used to determine an appropriate number of clusters.


In [ ]:
import pandas as pd
import numpy as np

data_path = 'data/2223RX19_Project_Data.xlsx'

# Load the xlsx file
df = pd.read_excel(data_path)

# split the categories into separate strings
categories = df['CATEGORY'].str.split(', ')

# create one-hot encoded vectors for each unique category
one_hot_categories = pd.get_dummies(categories.apply(pd.Series).stack()).sum(level=0)

# combine the one-hot encoded vectors with the original dataframe
df = pd.concat([df, one_hot_categories], axis=1)


In [ ]:
df

In [ ]:
from tqdm import tqdm
# determine the optimal number of clusters using the elbow method
step = 20
wcss = []
for i in tqdm(range(1, 600, step)):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(one_hot_categories)
    wcss.append(kmeans.inertia_)



In [ ]:
# plot the within-cluster sum of squares for different values of K
plt.plot(range(1, 600, step), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('Within-Cluster Sum of Squares')

# draw line at the elbow
plt.axvline(x=20, color='r', linestyle='--')

plt.show()

# Elbow methods indicates that the optimal number of clusters is 50 

kmeans = KMeans(n_clusters=8, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(one_hot_categories)

# add a new column to the original dataframe to store the cluster labels
df['cluster_label'] = kmeans.labels_

# export new df to xlsx 
df.to_excel('export/2223RX19_Project_Data_With_Cluster_Label.xlsx')


In [ ]:
# group the data by the new cluster labels and print the COMPANY_ID for each label
grouped = df.groupby('cluster_label')
for label, group in grouped:
    print(f'Cluster {label}: {list(group["COMPANY_ID"])}')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE


# apply t-SNE to reduce the dimensionality of the one-hot encoded categories
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(one_hot_categories)

# create a new dataframe with the t-SNE results
tsne_df = pd.DataFrame(data=tsne_results, columns=['tsne_1', 'tsne_2'])

# plot the t-SNE results and color the points by the cluster labels
plt.figure(figsize=(16,10))
plt.scatter(tsne_df['tsne_1'], tsne_df['tsne_2'], c=df['cluster_label'], cmap='tab20')
plt.title('t-SNE Clustering of Categories')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.show()